# Building Grade Exploration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pwd

'/home/viktor/Documents/electrify-chicago/src/data/research'

In [3]:
%cd ../../../

/home/viktor/Documents/electrify-chicago


/home/viktor/Documents/electrify-chicago/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
%pwd

'/home/viktor/Documents/electrify-chicago'

In [5]:
import pandas as pd
from pathlib import Path
import plotly.express as px

In [6]:
pd.set_option("display.float_format", "{:.4f}".format)

In [7]:
data_dir = Path("src/data/dist")

In [8]:
df = pd.read_csv(data_dir / "building-benchmarks.csv")

In [9]:
df

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,AvgPercentileLetterGrade,AvgPercentileLetterGradeQuintiles,GHGIntensityRankByPropertyType,TotalGHGEmissionsRankByPropertyType,ElectricityUseRankByPropertyType,NaturalGasUseRankByPropertyType,GrossFloorAreaRankByPropertyType,SourceEUIRankByPropertyType,SiteEUIRankByPropertyType,DataAnomalies
0,2022,100001,Presence SMEMC St Elizabeth Campus,Submitted Data,1431 N Claremont Ave,60622,NaN,True,WEST TOWN,Mixed Use Property,...,D,D,5.0000,14.0000,15.0000,9.0000,17.0000,5.0000,5.0000,NaN
1,2020,100002,Clemente Community Academy HS -CPS,Submitted Data,1147 N Western Ave,60622,2.0000,False,WEST TOWN,K-12 School,...,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,100019,Dixon Building,Submitted Data,411 - 415 S Wells St,60607,3.5000,False,LOOP,Office,...,B,C,224.5000,303.0000,306.0000,169.0000,315.0000,231.0000,145.0000,NaN
3,2022,100068,Joffco Square,Submitted Data,555 W Roosevelt Rd,60607,2.0000,False,NEAR WEST SIDE,Other - Mall,...,C,C,1.0000,2.0000,2.0000,3.0000,2.0000,1.0000,2.0000,gas:large-swings
4,2018,100148,The Jeffery Cyril Building,Submitted,7144-48 S Jeffery Blvd,60649,1.0000,False,SOUTH SHORE,Multifamily Housing,...,D,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,2022,260132,Q11970,Submitted Data,2401 W Grace St,60618,3.0000,False,NORTH CENTER,Office,...,B,C,130.0000,257.0000,245.0000,176.0000,265.0000,131.0000,150.0000,NaN
3388,2020,260135,NaN,Submitted Data,309 W 23RD ST,60616,4.0000,False,ARMOUR SQUARE,Multifamily Housing,...,B,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gas:large-swings
3389,2021,260149,Riverside Square & River's Edge/ 90033,Submitted,3145 S Ashland Ave,60608,4.0000,False,MCKINLEY PARK,Supermarket/Grocery Store,...,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gas:large-swings
3390,2019,260150,Cummings Life Science Center,Submitted,920 E 58th St,60637,2.0000,False,HYDE PARK,Laboratory,...,D,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# We drop these, because functions in this notebook were already used to generate these columns
# once in the .csv as part of the `process_data.py` script; so to start from scratch with this
# notebook, we should clean them out first:
grades_columns = [
    "AvgPercentileGrade",
    "AvgPercentileLetterGrade",
    "EnergyMixWeightedPctSum",
    "EnergyMixLetterGrade",
    "EnergyMixPercentileGrade",
    "GHGIntensityLetterGrade",
    "GHGIntensityPercentileGrade",
    "MissingRecordsCount",
    "SubmittedRecordsPercentileGrade",
    "SubmittedRecordsLetterGrade",
]
df.drop(
    grades_columns,
    axis=1,
    inplace=True,
)

In [11]:
latest_year = df["DataYear"].max()
latest_year

2022

In [12]:
df_current = df.loc[df["DataYear"] == latest_year]
df_current.head()

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,index,AvgPercentileLetterGradeQuintiles,GHGIntensityRankByPropertyType,TotalGHGEmissionsRankByPropertyType,ElectricityUseRankByPropertyType,NaturalGasUseRankByPropertyType,GrossFloorAreaRankByPropertyType,SourceEUIRankByPropertyType,SiteEUIRankByPropertyType,DataAnomalies
0,2022,100001,Presence SMEMC St Elizabeth Campus,Submitted Data,1431 N Claremont Ave,60622,NaN,True,WEST TOWN,Mixed Use Property,...,0,D,5.0000,14.0000,15.0000,9.0000,17.0000,5.0000,5.0000,NaN
2,2022,100019,Dixon Building,Submitted Data,411 - 415 S Wells St,60607,3.5000,False,LOOP,Office,...,2,C,224.5000,303.0000,306.0000,169.0000,315.0000,231.0000,145.0000,NaN
3,2022,100068,Joffco Square,Submitted Data,555 W Roosevelt Rd,60607,2.0000,False,NEAR WEST SIDE,Other - Mall,...,3,C,1.0000,2.0000,2.0000,3.0000,2.0000,1.0000,2.0000,gas:large-swings
6,2022,100211,NaN,Submitted Data,73 E LAKE ST,60601,1.5000,False,LOOP,Multifamily Housing,...,6,C,165.5000,194.0000,142.0000,299.0000,218.0000,158.5000,379.5000,gas:large-swings
7,2022,100252,NaN,Submitted Data,10426 S MICHIGAN AVE,60628,4.0000,False,ROSELAND,Multifamily Housing,...,7,A,1095.0000,1364.0000,1017.0000,1375.0000,1400.0000,1061.0000,1348.5000,NaN


In [13]:
print(f"There are {df_current.shape[0]} records for {latest_year}.")

There are 2549 records for 2022.


In [14]:
df.columns

Index(['DataYear', 'ID', 'PropertyName', 'ReportingStatus', 'Address',
       'ZIPCode', 'ChicagoEnergyRating', 'ExemptFromChicagoEnergyRating',
       'CommunityArea', 'PrimaryPropertyType', 'GrossFloorArea', 'YearBuilt',
       'NumberOfBuildings', 'WaterUse', 'ENERGYSTARScore', 'ElectricityUse',
       'NaturalGasUse', 'DistrictSteamUse', 'DistrictChilledWaterUse',
       'AllOtherFuelUse', 'SiteEUI', 'SourceEUI', 'WeatherNormalizedSiteEUI',
       'WeatherNormalizedSourceEUI', 'TotalGHGEmissions', 'GHGIntensity',
       'Latitude', 'Longitude', 'Location', 'Row_ID', 'Wards',
       'CommunityAreas', 'ZipCodes', 'CensusTracts',
       'HistoricalWards2003-2015', 'GHGIntensityRank',
       'GHGIntensityPercentileRank', 'TotalGHGEmissionsRank',
       'TotalGHGEmissionsPercentileRank', 'ElectricityUseRank',
       'ElectricityUsePercentileRank', 'NaturalGasUseRank',
       'NaturalGasUsePercentileRank', 'GrossFloorAreaRank',
       'GrossFloorAreaPercentileRank', 'SourceEUIRank',
    

## EDA Brainstorm

### Number of Records by Property Type

In [15]:
df_current["PrimaryPropertyType"].unique()

array(['Mixed Use Property', 'Office', 'Other - Mall',
       'Multifamily Housing', 'K-12 School',
       'Hospital (General Medical & Surgical)', 'Retail Store', 'Hotel',
       'Strip Mall', 'Senior Living Community', 'College/University',
       'Convention Center', 'Financial Office', 'Enclosed Mall',
       'Other - Entertainment/Public Assembly',
       'Fitness Center/Health Club/Gym', 'Medical Office', 'Other',
       'Library', 'Museum', 'Laboratory', 'Prison/Incarceration',
       'Other - Specialty Hospital', 'Supermarket/Grocery Store',
       'Wholesale Club/Supercenter', 'Residential Care Facility',
       'Movie Theater', 'Data Center', 'Other - Services',
       'Vehicle Dealership', 'Other - Recreation', 'Distribution Center',
       'Residence Hall/Dormitory', 'Performing Arts',
       'Other - Public Services', 'Worship Facility',
       'Pre-school/Daycare', 'Other - Lodging/Residential',
       'Urgent Care/Clinic/Other Outpatient', 'Other - Education',
       'Ou

In [16]:
count_by_prop_type = (
    df_current[["ID", "PrimaryPropertyType"]]
    .groupby("PrimaryPropertyType")
    .count()
    .reset_index()
)
count_by_prop_type

,PrimaryPropertyType,ID
0,Adult Education,2
1,College/University,67
2,Convention Center,2
3,Courthouse,1
4,Data Center,4
5,Distribution Center,1
6,Enclosed Mall,5
7,Financial Office,3
8,Fitness Center/Health Club/Gym,10
9,Hospital (General Medical & Surgical),18


In [17]:
count_by_prop_type.ID.min()

1

In [18]:
count_by_prop_type.ID.max()

1409

In [19]:
fig = px.bar(count_by_prop_type, x="PrimaryPropertyType", y="ID")

fig.update_layout(
    width=1200,
    height=600,
    title=f"Property Records Count by Primary Property Type for {latest_year}",
    yaxis_title="Count",
)

fig.show()

Conclusion: some categories are too sparse, so doing grading based on where a building stands compared to "peers" (same category) may be unreasonable. Unless we find data with corresponding categories but with more data points (e.g. national data).

## GHG Intensity Missingness and Distribution (Latest Year)

In [ ]:
fig = px.histogram(
    df_current,
    x="GHGIntensity",
    marginal="box",  # or violin, rug
    # hover_data=df.columns
)

# Calculate/graph mean
mean_value = df_current["GHGIntensity"].mean()
fig.add_vline(
    x=mean_value,
    line_width=2,
    line_dash="dash",
    # line_color="red",
    annotation_text="Mean",
    annotation_position="top left",
    annotation=dict(
        text=f"Mean: {mean_value:.2f}",
        showarrow=False,
        x=mean_value,
        y=1,
        yref="paper",
        align="left",
        # bgcolor="red",
        opacity=0.8,
    ),
)

fig.update_layout(
    width=1200,
    height=600,
    title=f"Distribution of GHG Intensity for {latest_year}",
)

fig.show()

In [21]:
num_ghg_missing_latest_year = df_current["GHGIntensity"].isna().sum()
print(
    f"Number of records without GHGIntensity in {latest_year}: {num_ghg_missing_latest_year}"
)

Number of records without GHGIntensity in 2022: 0


## GHG Intensity Distribution by Property Type (Latest Year)

In [22]:
fig = px.histogram(
    df_current,
    x="GHGIntensity",
    marginal="box",  # or violin, rug
    color="PrimaryPropertyType",
    # hover_data=df.columns
)


fig.update_layout(
    width=1200,
    height=600,
    title=f"Distribution of GHG Intensity for {latest_year} by Property Type",
)

fig.show()

## GHG Intensity vs Energy Star Rating (Latest Year)

In [23]:
missing_energy_star_num = df_current[["ENERGYSTARScore"]].isna().sum()
print(
    f"{missing_energy_star_num.iloc[0]} Energy Star Scores missing out of {df_current.shape[0]} for {latest_year}."
)

310 Energy Star Scores missing out of 2549 for 2022.


In [24]:
fig = px.scatter(
    df_current,
    y="GHGIntensity",
    x="ENERGYSTARScore",
    color="PrimaryPropertyType",
    hover_data=["ID", "GHGIntensity", "ENERGYSTARScore", "PrimaryPropertyType"],
)


fig.update_layout(
    width=1200,
    height=600,
    title=f"Energy Star Score vs. GHG Intensity for {latest_year}",
    xaxis_title="Energy Star Score (Higher is Better)",
    # hoverlabel=dict(
    #     bgcolor="white",
    #     font_size=16,
    #     font_family="Rockwell"
    # )
)

fig.show()

In [25]:
fig = px.scatter(
    df_current,
    y="GHGIntensity",
    x="ENERGYSTARScore",
    # color="PrimaryPropertyType",
    hover_data=[
        "ID",
        "GHGIntensity",
        "ENERGYSTARScore",
        # "PrimaryPropertyType"
    ],
)


fig.update_layout(
    width=1200,
    height=600,
    title=f"Energy Star Score vs. GHG Intensity for {latest_year}",
    xaxis_title="Energy Star Score (Higher is Better)",
    # hoverlabel=dict(
    #     bgcolor="white",
    #     font_size=16,
    #     font_family="Rockwell"
    # )
)

fig.show()

TODO: anomaly detection: new issue  
- across years; dramatic changes in GHG Intensity  
    - Flag anomalous data pre-grading  
    - by percentage change  
    - E.g. Maybe due to vacancy  
- 

In [26]:
correlation = df_current["ENERGYSTARScore"].corr(df_current["GHGIntensity"])

print(f"Correlation between ENERGYSTARScore and GHGIntensity: {correlation}")

Correlation between ENERGYSTARScore and GHGIntensity: -0.48459774903298447


Conclusion: Generally, higher Energy Star Score is (linearly) associated with lower GHG Intensity, as expected. But not all of GHG Intensity can be (linearly) explained by the Energy Star Score alone; about half is unaccounted for.

## Grading

### GHG Intensity

#### GHG Intensity Grade: Year-Based Percentile

Interpretation: Percentile grade of e.g. `56.3450` means that this building performs "better" than `56.3450%` of all buildings for the corresponding year.

Assign Grades:

In [27]:
from src.data.scripts.grade_buildings import generate_energy_int_grade

In [28]:
ghg_intensity_grades_df = generate_energy_int_grade(
    df=df, year=latest_year, cols_to_keep=["ID", "DataYear", "GHGIntensity"]
)
ghg_intensity_grades_df

,ID,DataYear,GHGIntensity,GHGIntensityPercentileGrade,GHGIntensityLetterGrade
0,100001,2022,13.0000,6.8262,F
2,100019,2022,6.0000,56.3750,C
3,100068,2022,13.9000,5.7670,F
6,100211,2022,8.1000,24.4802,D
7,100252,2022,4.5000,82.8953,A
...,...,...,...,...,...
3383,260118,2022,4.2000,87.0537,A
3385,260127,2022,5.9000,58.7681,C
3386,260129,2022,10.7000,11.0632,F
3387,260132,2022,7.3000,33.4641,D


##### Explore Grades:

###### GHG Intensity Percentile Grade Distribution

In [29]:
fig = px.histogram(
    ghg_intensity_grades_df,
    x="GHGIntensityPercentileGrade",
    marginal="box",  # or violin, rug
)

fig.update_layout(
    width=1200,
    height=600,
    title=f"Distribution of GHG Intensity Percentile Grades for {latest_year}",
)

fig.show()

##### GHG Intensity Percentile Grade vs GHG Intensity

In [30]:
fig = px.scatter(
    ghg_intensity_grades_df,
    x="GHGIntensity",
    y="GHGIntensityPercentileGrade",
    color="GHGIntensityLetterGrade",
)

fig.update_layout(
    width=1200,
    height=600,
    title=f"GHG Intensity Percentile Grade vs. GHG Intensity for {latest_year}",
)

fig.show()
# TODO: link via ID in the tooltip or just by ID with `https://electrifychicago.net/building-id/:buildingId`

##### Grade GHG Intensity for All Years

In [31]:
from src.data.scripts.grade_buildings import apply_grade_func_all_years

In [32]:
ghg_intensity_grades_all_years_df = apply_grade_func_all_years(
    df=df, func=generate_energy_int_grade
)
ghg_intensity_grades_all_years_df

,ID,DataYear,GHGIntensityPercentileGrade,GHGIntensityLetterGrade
0,100001,2022,6.8262,F
2,100019,2022,56.3750,C
3,100068,2022,5.7670,F
6,100211,2022,24.4802,D
7,100252,2022,82.8953,A
...,...,...,...,...
2271,251313,2016,62.9630,B
2297,251341,2016,22.2222,D
2312,251364,2016,29.6296,D
2923,253995,2016,66.6667,B


In [33]:
df.merge(ghg_intensity_grades_all_years_df, on=["ID", "DataYear"])

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,GHGIntensityRankByPropertyType,TotalGHGEmissionsRankByPropertyType,ElectricityUseRankByPropertyType,NaturalGasUseRankByPropertyType,GrossFloorAreaRankByPropertyType,SourceEUIRankByPropertyType,SiteEUIRankByPropertyType,DataAnomalies,GHGIntensityPercentileGrade,GHGIntensityLetterGrade
0,2022,100001,Presence SMEMC St Elizabeth Campus,Submitted Data,1431 N Claremont Ave,60622,NaN,True,WEST TOWN,Mixed Use Property,...,5.0000,14.0000,15.0000,9.0000,17.0000,5.0000,5.0000,NaN,6.8262,F
1,2020,100002,Clemente Community Academy HS -CPS,Submitted Data,1147 N Western Ave,60622,2.0000,False,WEST TOWN,K-12 School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.6246,F
2,2022,100019,Dixon Building,Submitted Data,411 - 415 S Wells St,60607,3.5000,False,LOOP,Office,...,224.5000,303.0000,306.0000,169.0000,315.0000,231.0000,145.0000,NaN,56.3750,C
3,2022,100068,Joffco Square,Submitted Data,555 W Roosevelt Rd,60607,2.0000,False,NEAR WEST SIDE,Other - Mall,...,1.0000,2.0000,2.0000,3.0000,2.0000,1.0000,2.0000,gas:large-swings,5.7670,F
4,2018,100148,The Jeffery Cyril Building,Submitted,7144-48 S Jeffery Blvd,60649,1.0000,False,SOUTH SHORE,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.9130,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,2022,260132,Q11970,Submitted Data,2401 W Grace St,60618,3.0000,False,NORTH CENTER,Office,...,130.0000,257.0000,245.0000,176.0000,265.0000,131.0000,150.0000,NaN,33.4641,D
3388,2020,260135,NaN,Submitted Data,309 W 23RD ST,60616,4.0000,False,ARMOUR SQUARE,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gas:large-swings,48.1728,C
3389,2021,260149,Riverside Square & River's Edge/ 90033,Submitted,3145 S Ashland Ave,60608,4.0000,False,MCKINLEY PARK,Supermarket/Grocery Store,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gas:large-swings,8.6466,F
3390,2019,260150,Cummings Life Science Center,Submitted,920 E 58th St,60637,2.0000,False,HYDE PARK,Laboratory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.8182,F


##### Limitations of this grading method:

- Is not a peer-to-peer comparison; e.g. a 24/7 hospital would be graded with the same methodology as an office building.
- Even if all buildings as a whole emit less and less GHG year-over-year, there will always be the same proportion of them in each letter grade category within each given year. I.e. there will always be 1/5 of building receiving each particular grade within each year => should there be instead a reference (index/benchmark) year? Or a different grading methodology that accounts for improvement over time?
- An alternative: do similar to Energy Star Scores: calculate a building's expected GHG Intensity via OLS linear regression based on its peers, then do ratio between actual GHG Intensity and its predicted value, then do percentile score. OLS can be run using [Building Performance Database (BPD) Application Programming Interface (API)](https://buildings.lbl.gov/cbs/bpd-api-documentation/).

### Energy Mix

#### Energy Mix Grade: Simple Arbitrary Weight Score

Interpretation: Percentile grade of e.g. `56.3450` means that this building performs "better" than `56.3450%` of all buildings for the corresponding year. The score used for the Percentile Grade calculation is a weighted sum of percentages of each energy source for a corresponding building. 

In [34]:
from src.data.scripts.grade_buildings import generate_energymix_grade

In [35]:
energymix_cols_to_keep = [
    "ID",
    "DataYear",
    "ElectricityUse",
    "NaturalGasUse",
    "DistrictSteamUse",
    "DistrictChilledWaterUse",
    "AllOtherFuelUse",
]
energymix_grades_df = generate_energymix_grade(
    df, year=2022, cols_to_keep=energymix_cols_to_keep
)
energymix_grades_df

,ID,DataYear,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade
0,100001,2022,16795066.2000,33899078.5000,0.0000,0.0000,NaN,33.1302,45.2727,C
2,100019,2022,1641381.8000,2465232.2000,0.0000,0.0000,NaN,39.9692,56.5712,C
3,100068,2022,8354489.4000,331613.0000,0.0000,0.0000,NaN,96.1823,92.7815,A
6,100211,2022,12253233.4000,14931549.6000,0.0000,0.0000,NaN,45.0739,64.0251,B
7,100252,2022,1550327.6000,110041.9000,0.0000,0.0000,NaN,93.3724,92.3499,A
...,...,...,...,...,...,...,...,...,...,...
3383,260118,2022,669935.5000,3679236.0000,0.0000,0.0000,NaN,15.4037,9.1016,F
3385,260127,2022,7924051.9000,3717569.9000,0.0000,0.0000,NaN,68.0666,85.2883,A
3386,260129,2022,2750911.1000,3444364.9000,0.0000,0.0000,NaN,44.4034,63.2797,B
3387,260132,2022,4204386.8000,2280683.7000,0.0000,0.0000,NaN,64.8318,83.8760,A


##### Explore Grades:

In [36]:
# TODO

##### Grade All Years for Energy Mix Grade:

In [37]:
energy_mix_grades_all_years_df = apply_grade_func_all_years(
    df=df, func=generate_energymix_grade
)
energy_mix_grades_all_years_df

,ID,DataYear,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade
0,100001,2022,33.1302,45.2727,C
2,100019,2022,39.9692,56.5712,C
3,100068,2022,96.1823,92.7815,A
6,100211,2022,45.0739,64.0251,B
7,100252,2022,93.3724,92.3499,A
...,...,...,...,...,...
2271,251313,2016,8.9737,18.5185,F
2297,251341,2016,87.9599,88.8889,A
2312,251364,2016,40.8164,55.5556,C
2923,253995,2016,54.5816,70.3704,B


In [38]:
df.merge(energy_mix_grades_all_years_df, on=["ID", "DataYear"])

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,TotalGHGEmissionsRankByPropertyType,ElectricityUseRankByPropertyType,NaturalGasUseRankByPropertyType,GrossFloorAreaRankByPropertyType,SourceEUIRankByPropertyType,SiteEUIRankByPropertyType,DataAnomalies,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade
0,2022,100001,Presence SMEMC St Elizabeth Campus,Submitted Data,1431 N Claremont Ave,60622,NaN,True,WEST TOWN,Mixed Use Property,...,14.0000,15.0000,9.0000,17.0000,5.0000,5.0000,NaN,33.1302,45.2727,C
1,2020,100002,Clemente Community Academy HS -CPS,Submitted Data,1147 N Western Ave,60622,2.0000,False,WEST TOWN,K-12 School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.2533,93.3555,A
2,2022,100019,Dixon Building,Submitted Data,411 - 415 S Wells St,60607,3.5000,False,LOOP,Office,...,303.0000,306.0000,169.0000,315.0000,231.0000,145.0000,NaN,39.9692,56.5712,C
3,2022,100068,Joffco Square,Submitted Data,555 W Roosevelt Rd,60607,2.0000,False,NEAR WEST SIDE,Other - Mall,...,2.0000,2.0000,3.0000,2.0000,1.0000,2.0000,gas:large-swings,96.1823,92.7815,A
4,2018,100148,The Jeffery Cyril Building,Submitted,7144-48 S Jeffery Blvd,60649,1.0000,False,SOUTH SHORE,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.3895,10.8696,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,2022,260132,Q11970,Submitted Data,2401 W Grace St,60618,3.0000,False,NORTH CENTER,Office,...,257.0000,245.0000,176.0000,265.0000,131.0000,150.0000,NaN,64.8318,83.8760,A
3388,2020,260135,NaN,Submitted Data,309 W 23RD ST,60616,4.0000,False,ARMOUR SQUARE,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,gas:large-swings,88.7433,92.3588,A
3389,2021,260149,Riverside Square & River's Edge/ 90033,Submitted,3145 S Ashland Ave,60608,4.0000,False,MCKINLEY PARK,Supermarket/Grocery Store,...,NaN,NaN,NaN,NaN,NaN,NaN,gas:large-swings,61.0021,78.1955,B
3390,2019,260150,Cummings Life Science Center,Submitted,920 E 58th St,60637,2.0000,False,HYDE PARK,Laboratory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.5689,67.2727,B


#### [WIP] Energy Mix Grade: Score Weighted Using GHG Emission Factor 

Interpretation: Same as the Simple Arbitrary Weight Score above, except weights are calculated off of the GHG Emission factos used by Energy Star.

These data are from [`Energy Star's Historical Greenhouse Gas Factors, 2000 – present`](https://www.energystar.gov/buildings/tools-and-resources/historical-greenhouse-gas-factors-2000-present). When the `Subregion` is `NaN`, Energy Uses the national factor (i.e. no division by subregion for those energy sources).

In [39]:
data_dir

PosixPath('src/data/dist')

In [43]:
basic_rows_to_skip = [0, 1, 2]

direct_ghg_emissions_factors_df = pd.read_excel(
    "src/data/research/data/Historical_GHG_Factors.xlsx",
    sheet_name="US Emission Factors",
    usecols="A:AA",
    skiprows=basic_rows_to_skip + [4],
    nrows=11,
)

direct_ghg_emissions_factors_df

,Fuel Type,Subregion,2000,2001,2002,2003,2004,2005,2006,2007,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Natural Gas,NaN,53.0720,53.0720,53.0720,53.0720,53.0720,53.0720,53.0720,53.0720,...,53.1148,53.1148,53.1148,53.1148,53.1148,53.1148,53.1148,53.1145,53.1145,53.1145
1,Propane,NaN,61.4956,61.4956,61.4956,61.4956,61.4956,61.4956,61.4956,61.4956,...,64.2480,64.2480,64.2480,64.2480,64.2480,64.2480,64.2480,61.9530,61.9530,61.9530
2,Fuel Oil (No. 1),NaN,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,...,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,73.4930,73.4930,73.4930
3,Fuel Oil (No. 2),NaN,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,...,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2030,74.2030,74.2030
4,Fuel Oil (No. 4),NaN,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,...,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,75.2830,75.2830,75.2830
5,Fuel Oil (No. 5 & No. 6),NaN,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,...,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,74.2580,74.2580,74.2580
6,Diesel,NaN,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,...,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,75.1630,75.1630,75.1630
7,Coal (anthracite),NaN,104.2670,104.2670,104.2670,104.2670,104.2670,104.2670,104.2670,104.2670,...,104.4418,104.4418,104.4418,104.4418,104.4418,104.4418,104.4418,104.4220,104.4220,104.4220
8,Coal (bituminous),NaN,94.1270,94.1270,94.1270,94.1270,94.1270,94.1270,94.1270,94.1270,...,94.0318,94.0318,94.0318,94.0318,94.0318,94.0318,94.0318,94.0120,94.0120,94.0120
9,Coke,NaN,102.7670,102.7670,102.7670,102.7670,102.7670,102.7670,102.7670,102.7670,...,114.4218,114.4218,114.4218,114.4218,114.4218,114.4218,114.4218,114.4020,114.4020,114.4020


In [44]:
indirect_ghg_emissions_district_fuels_df = pd.read_excel(
    "src/data/research/data/Historical_GHG_Factors.xlsx",
    sheet_name="US Emission Factors",
    usecols="A:AA",
    skiprows=basic_rows_to_skip + list(range(basic_rows_to_skip[-1] + 2, 17)),
    nrows=5,
)

indirect_ghg_emissions_district_fuels_df

,Fuel Type,Subregion,2000,2001,2002,2003,2004,2005,2006,2007,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,District Steam,NaN,88.5385,88.5385,88.5385,88.5385,88.5385,88.5385,88.5385,88.5385,...,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985
1,District Hot Water,NaN,88.5385,88.5385,88.5385,88.5385,88.5385,88.5385,88.5385,88.5385,...,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985,66.3985
2,District Chilled Water - Electric Driven Chiller,NaN,52.7047,52.7047,52.7047,52.7047,52.7047,52.7047,52.7047,52.7047,...,52.7004,52.7004,52.7004,52.7004,52.7004,52.7004,52.7004,52.7004,52.7004,52.7004
3,District Chilled Water - Absorption Chiller us...,NaN,73.8636,73.8636,73.8636,73.8636,73.8636,73.8636,73.8636,73.8636,...,73.8893,73.8893,73.8893,73.8893,73.8893,73.8893,73.8893,73.8893,73.8893,73.8893
4,District Chilled Water - Engine-Driven Chiller...,NaN,49.2905,49.2905,49.2905,49.2905,49.2905,49.2905,49.2905,49.2905,...,49.3082,49.3082,49.3082,49.3082,49.3082,49.3082,49.3082,49.3082,49.3082,49.3082


In [45]:
biomass_ghg_emissions_district_fuels_df = pd.read_excel(
    "src/data/research/data/Historical_GHG_Factors.xlsx",
    sheet_name="US Emission Factors",
    usecols="A:AA",
    skiprows=basic_rows_to_skip + list(range(basic_rows_to_skip[-1] + 2, 23)),
    nrows=1,
)

biomass_ghg_emissions_district_fuels_df

,Fuel Type,Subregion,2000,2001,2002,2003,2004,2005,2006,2007,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Wood,NaN,95.7740,95.7740,95.7740,95.7740,95.7740,95.7740,95.7740,95.7740,...,95.0528,94.2227,94.2227,95.0528,95.0528,95.0528,95.0528,94.9556,94.9556,94.9556


In [46]:
# Chicago is in RFCW subregion, based on https://portfoliomanager.energystar.gov/pdf/reference/Emissions.pdf
indirect_ghg_emissions_electricity_df = pd.read_excel(
    "src/data/research/data/Historical_GHG_Factors.xlsx",
    sheet_name="US Emission Factors",
    usecols="A:AA",
    skiprows=basic_rows_to_skip + list(range(basic_rows_to_skip[-1] + 2, 25)),
    nrows=28,
)
indirect_ghg_emissions_electricity_df = indirect_ghg_emissions_electricity_df.loc[
    indirect_ghg_emissions_electricity_df["Subregion"] == "RFCW"
]
indirect_ghg_emissions_electricity_df

,Fuel Type,Subregion,2000,2001,2002,2003,2004,2005,2006,2007,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
18,Electricity - Grid,RFCW,205.5095,205.5095,205.5095,205.5095,205.5095,205.5095,205.5095,207.3702,...,184.8948,166.3642,166.3642,156.0693,142.8265,131.7142,139.9196,133.7058,133.7058,133.7058


In [47]:
# Combine all GHG Emission factors into a single DF for ease of handling:
emission_factors_dfs = [
    direct_ghg_emissions_factors_df,
    indirect_ghg_emissions_district_fuels_df,
    biomass_ghg_emissions_district_fuels_df,
    indirect_ghg_emissions_electricity_df,
]
emission_factors_df = pd.concat(emission_factors_dfs).reset_index(drop=True)
emission_factors_df

,Fuel Type,Subregion,2000,2001,2002,2003,2004,2005,2006,2007,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Natural Gas,NaN,53.0720,53.0720,53.0720,53.0720,53.0720,53.0720,53.0720,53.0720,...,53.1148,53.1148,53.1148,53.1148,53.1148,53.1148,53.1148,53.1145,53.1145,53.1145
1,Propane,NaN,61.4956,61.4956,61.4956,61.4956,61.4956,61.4956,61.4956,61.4956,...,64.2480,64.2480,64.2480,64.2480,64.2480,64.2480,64.2480,61.9530,61.9530,61.9530
2,Fuel Oil (No. 1),NaN,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,...,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,73.4990,73.4930,73.4930,73.4930
3,Fuel Oil (No. 2),NaN,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,...,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2030,74.2030,74.2030
4,Fuel Oil (No. 4),NaN,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,...,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,75.2890,75.2830,75.2830,75.2830
5,Fuel Oil (No. 5 & No. 6),NaN,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,...,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,75.3490,74.2580,74.2580,74.2580
6,Diesel,NaN,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,...,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,74.2090,75.1630,75.1630,75.1630
7,Coal (anthracite),NaN,104.2670,104.2670,104.2670,104.2670,104.2670,104.2670,104.2670,104.2670,...,104.4418,104.4418,104.4418,104.4418,104.4418,104.4418,104.4418,104.4220,104.4220,104.4220
8,Coal (bituminous),NaN,94.1270,94.1270,94.1270,94.1270,94.1270,94.1270,94.1270,94.1270,...,94.0318,94.0318,94.0318,94.0318,94.0318,94.0318,94.0318,94.0120,94.0120,94.0120
9,Coke,NaN,102.7670,102.7670,102.7670,102.7670,102.7670,102.7670,102.7670,102.7670,...,114.4218,114.4218,114.4218,114.4218,114.4218,114.4218,114.4218,114.4020,114.4020,114.4020


In [48]:
# Check that there is nothing in the unknown fuel category; so that we ignore this fuel source in this grade's calculation:
assert df.AllOtherFuelUse.isna().sum() == df.shape[0]

In [49]:
emission_factors_latest_year: pd.DataFrame = emission_factors_df.loc[
    :,
    ["Fuel Type", str(latest_year)],
]
emission_factors_latest_year

,Fuel Type,2022
0,Natural Gas,53.1145
1,Propane,61.9530
2,Fuel Oil (No. 1),73.4930
3,Fuel Oil (No. 2),74.2030
4,Fuel Oil (No. 4),75.2830
5,Fuel Oil (No. 5 & No. 6),74.2580
6,Diesel,75.1630
7,Coal (anthracite),104.4220
8,Coal (bituminous),94.0120
9,Coke,114.4020


In [50]:
# Since the building dataset doesn't specify what kind of District Chilled Water is used, take the mean of all three types:
chilled_water_title = "District Chilled Water"
chilled_water_filter = emission_factors_latest_year["Fuel Type"].str.contains(
    chilled_water_title
)
chilled_water_factor_mean = emission_factors_latest_year.loc[
    chilled_water_filter, str(latest_year)
].mean()
chilled_water_factor_mean

58.632611112222236

In [51]:
chilled_water_row = {
    "Fuel Type": chilled_water_title,
    str(latest_year): chilled_water_factor_mean,
}
chilled_water_row

{'Fuel Type': 'District Chilled Water', '2022': 58.632611112222236}

In [52]:
chilled_water_row_df = pd.DataFrame([chilled_water_row])
chilled_water_row_df

,Fuel Type,2022
0,District Chilled Water,58.6326


In [53]:
emission_factors_processed = emission_factors_latest_year.loc[~chilled_water_filter]
emission_factors_processed = pd.concat(
    [emission_factors_processed, chilled_water_row_df], ignore_index=True
)
emission_factors_processed

,Fuel Type,2022
0,Natural Gas,53.1145
1,Propane,61.9530
2,Fuel Oil (No. 1),73.4930
3,Fuel Oil (No. 2),74.2030
4,Fuel Oil (No. 4),75.2830
5,Fuel Oil (No. 5 & No. 6),74.2580
6,Diesel,75.1630
7,Coal (anthracite),104.4220
8,Coal (bituminous),94.0120
9,Coke,114.4020


In [54]:
# Map column names from the buildings dataset to the fuel type:
energy_use_to_factors_map = {
    "ElectricityUse": "Electricity - Grid",
    "NaturalGasUse": "Natural Gas",
    "DistrictSteamUse": "District Steam",
    "DistrictChilledWaterUse": "District Chilled Water",
}
energy_use_to_factors_map.keys()

dict_keys(['ElectricityUse', 'NaturalGasUse', 'DistrictSteamUse', 'DistrictChilledWaterUse'])

In [55]:
emission_factors_processed.loc[
    emission_factors_processed["Fuel Type"].isin(list(energy_use_to_factors_map.keys()))
]

,Fuel Type,2022


In [56]:
# Convert emission factors to weights:
# TODO

### Generate Grades for Both GHG Intensity and Energy Mix for All Years

In [57]:
from src.data.scripts.grade_buildings import grade_ghg_intensity_energy_mix_all_years

In [58]:
grade_ghg_intensity_energy_mix_all_years(building_data=df)

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,NaturalGasUseRankByPropertyType,GrossFloorAreaRankByPropertyType,SourceEUIRankByPropertyType,SiteEUIRankByPropertyType,DataAnomalies,GHGIntensityPercentileGrade,GHGIntensityLetterGrade,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade
0,2022,100001,Presence SMEMC St Elizabeth Campus,Submitted Data,1431 N Claremont Ave,60622,NaN,True,WEST TOWN,Mixed Use Property,...,9.0000,17.0000,5.0000,5.0000,NaN,6.8262,F,33.1302,45.2727,C
1,2020,100002,Clemente Community Academy HS -CPS,Submitted Data,1147 N Western Ave,60622,2.0000,False,WEST TOWN,K-12 School,...,NaN,NaN,NaN,NaN,NaN,12.6246,F,93.2533,93.3555,A
2,2022,100019,Dixon Building,Submitted Data,411 - 415 S Wells St,60607,3.5000,False,LOOP,Office,...,169.0000,315.0000,231.0000,145.0000,NaN,56.3750,C,39.9692,56.5712,C
3,2022,100068,Joffco Square,Submitted Data,555 W Roosevelt Rd,60607,2.0000,False,NEAR WEST SIDE,Other - Mall,...,3.0000,2.0000,1.0000,2.0000,gas:large-swings,5.7670,F,96.1823,92.7815,A
4,2018,100148,The Jeffery Cyril Building,Submitted,7144-48 S Jeffery Blvd,60649,1.0000,False,SOUTH SHORE,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,23.9130,D,12.3895,10.8696,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,2022,260132,Q11970,Submitted Data,2401 W Grace St,60618,3.0000,False,NORTH CENTER,Office,...,176.0000,265.0000,131.0000,150.0000,NaN,33.4641,D,64.8318,83.8760,A
3388,2020,260135,NaN,Submitted Data,309 W 23RD ST,60616,4.0000,False,ARMOUR SQUARE,Multifamily Housing,...,NaN,NaN,NaN,NaN,gas:large-swings,48.1728,C,88.7433,92.3588,A
3389,2021,260149,Riverside Square & River's Edge/ 90033,Submitted,3145 S Ashland Ave,60608,4.0000,False,MCKINLEY PARK,Supermarket/Grocery Store,...,NaN,NaN,NaN,NaN,gas:large-swings,8.6466,F,61.0021,78.1955,B
3390,2019,260150,Cummings Life Science Center,Submitted,920 E 58th St,60637,2.0000,False,HYDE PARK,Laboratory,...,NaN,NaN,NaN,NaN,NaN,1.8182,F,42.5689,67.2727,B


### Reporting Years (not reporting is bad)

In [59]:
df_historical = pd.read_csv(data_dir / "benchmarking-all-years.csv")

In [60]:
df_historical.loc[:, "ReportingStatus"].unique()

array(['Submitted Data', 'Not Submitted', 'Exempt', 'Submitted'],
      dtype=object)

In [61]:
df_historical_non_exempt = df_historical.loc[
    df_historical["ReportingStatus"] != "Exempt"
]

In [62]:
df_historical.loc[:, "ReportingStatus"].isna().sum()

0

In [63]:
df_historical.columns

Index(['ID', 'DataYear', 'ReportingStatus', 'GrossFloorArea',
       'TotalGHGEmissions', 'GHGIntensity', 'NumberOfBuildings',
       'ChicagoEnergyRating', 'ENERGYSTARScore', 'ElectricityUse',
       'NaturalGasUse', 'DistrictSteamUse', 'DistrictChilledWaterUse',
       'AllOtherFuelUse', 'SiteEUI', 'SourceEUI', 'WeatherNormalizedSiteEUI',
       'WeatherNormalizedSourceEUI'],
      dtype='object')

In [64]:
df_historical.loc[:, "DataYear"].unique()

array([2020, 2018, 2015, 2019, 2016, 2017, 2014, 2021, 2022])

In [65]:
df_historical

,ID,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI
0,252064,2020,Submitted Data,64028.0000,1160.9000,18.1000,1.0000,2.0000,NaN,2384738.9000,NaN,7438787.0000,5594040.1000,NaN,240.8000,323.6000,246.0000,329.9000
1,232458,2020,Submitted Data,627680.0000,4871.7000,7.8000,1.0000,1.0000,22.0000,16397682.8000,43537490.6000,NaN,NaN,NaN,95.5000,146.0000,100.3000,150.7000
2,254616,2020,Submitted Data,555524.0000,4581.4000,8.2000,1.0000,2.0000,49.0000,28606427.7000,2199940.1000,NaN,NaN,NaN,55.5000,148.3000,56.7000,151.8000
3,103812,2020,Submitted Data,130007.0000,1092.1000,8.4000,1.0000,3.0000,61.0000,6489281.3000,1493523.2000,NaN,NaN,NaN,61.4000,151.8000,63.0000,154.8000
4,254073,2020,Submitted Data,83000.0000,295.8000,3.6000,1.0000,4.0000,100.0000,1614582.3000,825006.6000,NaN,NaN,NaN,29.4000,64.9000,29.6000,64.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24886,251598,2022,Submitted Data,77483.0000,592.6000,7.6000,1.0000,1.0000,25.0000,2544345.5000,4847890.5000,0.0000,0.0000,NaN,95.4000,157.6000,99.3000,161.8000
24887,159799,2022,Submitted Data,150000.0000,651.3000,4.3000,1.0000,4.0000,85.0000,2019564.1000,7254640.6000,0.0000,0.0000,NaN,61.8000,88.5000,64.5000,91.3000
24888,105488,2022,Submitted Data,128387.0000,1034.4000,8.4000,1.0000,2.0000,25.0000,2801125.0000,12528588.7000,0.0000,0.0000,NaN,124.3000,170.3000,129.9000,175.2000
24889,251565,2022,Submitted Data,101627.0000,345.2000,3.4000,1.0000,4.0000,91.0000,733952.0000,4679269.9000,0.0000,0.0000,NaN,53.3000,68.6000,55.2000,69.7000


##### Grade All Years for Reporting Status:

In [67]:
from src.data.scripts.grade_buildings import generate_consistent_reporting_grade

In [68]:
not_submitted_count_df = generate_consistent_reporting_grade(df_historical)
not_submitted_count_df

,index,ID,SubmittedRecordsPercentileGrade,MissingRecordsCount,SubmittedRecordsLetterGrade
0,0,100001,100.0000,0.0000,A
1,1,100002,80.0000,1.0000,A
2,2,100019,100.0000,0.0000,A
3,3,100068,100.0000,0.0000,A
4,4,100148,42.8571,4.0000,C
...,...,...,...,...,...
3744,3744,260132,100.0000,0.0000,A
3745,3745,260135,85.7143,1.0000,A
3746,3746,260149,100.0000,0.0000,A
3747,3747,260150,100.0000,0.0000,A


### Overall Grades

In [69]:
grades_df = grade_ghg_intensity_energy_mix_all_years(building_data=df)
grades_df = pd.merge(
    left=grades_df,
    right=not_submitted_count_df,
    how="left",
    on="ID",
)
grades_df["AvgPercentileGrade"] = grades_df[
    [
        "GHGIntensityPercentileGrade",
        "EnergyMixPercentileGrade",
        "SubmittedRecordsPercentileGrade",
    ]
].mean(axis=1)

In [70]:
grades_df

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,GHGIntensityPercentileGrade,GHGIntensityLetterGrade,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade,index_y,SubmittedRecordsPercentileGrade,MissingRecordsCount,SubmittedRecordsLetterGrade,AvgPercentileGrade
0,2022,100001,Presence SMEMC St Elizabeth Campus,Submitted Data,1431 N Claremont Ave,60622,NaN,True,WEST TOWN,Mixed Use Property,...,6.8262,F,33.1302,45.2727,C,0,100.0000,0.0000,A,50.6996
1,2020,100002,Clemente Community Academy HS -CPS,Submitted Data,1147 N Western Ave,60622,2.0000,False,WEST TOWN,K-12 School,...,12.6246,F,93.2533,93.3555,A,1,80.0000,1.0000,A,61.9934
2,2022,100019,Dixon Building,Submitted Data,411 - 415 S Wells St,60607,3.5000,False,LOOP,Office,...,56.3750,C,39.9692,56.5712,C,2,100.0000,0.0000,A,70.9821
3,2022,100068,Joffco Square,Submitted Data,555 W Roosevelt Rd,60607,2.0000,False,NEAR WEST SIDE,Other - Mall,...,5.7670,F,96.1823,92.7815,A,3,100.0000,0.0000,A,66.1828
4,2018,100148,The Jeffery Cyril Building,Submitted,7144-48 S Jeffery Blvd,60649,1.0000,False,SOUTH SHORE,Multifamily Housing,...,23.9130,D,12.3895,10.8696,F,4,42.8571,4.0000,C,25.8799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,2022,260132,Q11970,Submitted Data,2401 W Grace St,60618,3.0000,False,NORTH CENTER,Office,...,33.4641,D,64.8318,83.8760,A,3744,100.0000,0.0000,A,72.4467
3388,2020,260135,NaN,Submitted Data,309 W 23RD ST,60616,4.0000,False,ARMOUR SQUARE,Multifamily Housing,...,48.1728,C,88.7433,92.3588,A,3745,85.7143,1.0000,A,75.4153
3389,2021,260149,Riverside Square & River's Edge/ 90033,Submitted,3145 S Ashland Ave,60608,4.0000,False,MCKINLEY PARK,Supermarket/Grocery Store,...,8.6466,F,61.0021,78.1955,B,3746,100.0000,0.0000,A,62.2807
3390,2019,260150,Cummings Life Science Center,Submitted,920 E 58th St,60637,2.0000,False,HYDE PARK,Laboratory,...,1.8182,F,42.5689,67.2727,B,3747,100.0000,0.0000,A,56.3636


In [71]:
fig = px.histogram(
    grades_df,
    x="AvgPercentileGrade",
)

fig.update_layout(
    width=1200,
    height=600,
    title="Distribution of Overall Numerical Grades for All Years",
)

fig.show()

In [72]:
from src.data.scripts.grade_buildings import bins, letter_grades

In [73]:
grades_df["AvgPercentileLetterGrade"] = pd.cut(
    grades_df["AvgPercentileGrade"], bins=bins, labels=letter_grades, right=True
)

In [85]:
fig = px.histogram(
    grades_df,
    "AvgPercentileLetterGrade",
)

fig.update_layout(
    width=1200,
    height=600,
    title="Distribution of Overall Letter Grades for All Years",
    xaxis={"categoryorder": "category ascending"},
)

fig.show()

# Colton Overall Grades Anlaysis

In [75]:
grades_df["ElectricityMix"] = round(
    100
    * (
        grades_df["ElectricityUse"].fillna(0)
        / (
            grades_df["ElectricityUse"].fillna(0)
            + grades_df["NaturalGasUse"].fillna(0)
            + grades_df["DistrictSteamUse"].fillna(0)
            + grades_df["DistrictChilledWaterUse"].fillna(0)
            + grades_df["AllOtherFuelUse"].fillna(0)
        )
    ),
    1,
)

cols_deciding_grade = ["GHGIntensity", "ElectricityMix", "MissingRecordsCount"]
id_cols = ["DataYear", "ID", "PropertyName"]
outcome_col = "AvgPercentileLetterGrade"

grades_df[cols_deciding_grade + id_cols + [outcome_col]]

,GHGIntensity,ElectricityMix,MissingRecordsCount,DataYear,ID,PropertyName,AvgPercentileLetterGrade
0,13.0000,33.1000,0.0000,2022,100001,Presence SMEMC St Elizabeth Campus,C
1,10.4000,93.3000,1.0000,2020,100002,Clemente Community Academy HS -CPS,B
2,6.0000,40.0000,0.0000,2022,100019,Dixon Building,B
3,13.9000,96.2000,0.0000,2022,100068,Joffco Square,B
4,10.7000,12.4000,4.0000,2018,100148,The Jeffery Cyril Building,D
...,...,...,...,...,...,...,...
3387,7.3000,64.8000,0.0000,2022,260132,Q11970,B
3388,6.4000,88.7000,1.0000,2020,260135,NaN,B
3389,19.0000,61.0000,0.0000,2021,260149,Riverside Square & River's Edge/ 90033,B
3390,45.3000,42.6000,0.0000,2019,260150,Cummings Life Science Center,C


In [76]:
import plotly.express as px

# Define color mapping for letter grades
color_map = {
    "A": "#00FF00",  # Bright green
    "B": "yellowgreen",
    "C": "yellow",
    "D": "orange",
    "F": "red",
}


# Create a 3D scatter plot
fig = px.scatter_3d(
    grades_df,
    x="GHGIntensity",
    y="ElectricityMix",
    z="MissingRecordsCount",
    color="AvgPercentileLetterGrade",
    color_discrete_map=color_map,
    hover_data=[
        "PropertyName",
        "DataYear",
        "ID",
        "GHGIntensity",
        "ElectricityMix",
        "MissingRecordsCount",
    ],
    category_orders={
        "AvgPercentileLetterGrade": ["A", "B", "C", "D", "F"]
    },  # Set custom legend order
)


# Set axis limits and adjust figure size
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[0, 50])  # Limit GHGIntensity axis from 0 to 120
    ),
    width=1200,  # Adjust width (default is ~700)
    height=800,  # Adjust height (default is ~450)
)


# Show the plot
fig.show()

In [90]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np


def create_horizontal_subplots(
    df, columns_info, group_col, group_color_mapping, hover_data
):
    """
    Creates multiple horizontal subplots (stacked). Each row is a scatter plot
    of x vs. a random jitter, colored by group.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe containing the columns to be plotted and group_col.
    columns_info : list of dict
        Each dict should have:
          {
            "col": <column name in df>,
            "range": [min_val, max_val],
            "reverse": True/False
          }
    group_col : str
        Name of the column in df used for grouping/coloring.
    group_color_mapping : dict
        Maps each group label to a color, e.g. {"Group A": "blue", ...}
    """

    n_plots = len(columns_info)
    fig = make_subplots(
        rows=n_plots,
        cols=1,
        shared_xaxes=False,  # each plot can have its own axis range
        subplot_titles=[info["col"] for info in columns_info],
        vertical_spacing=0.07,
    )

    # We'll create the subplots row by row
    for i, info in enumerate(columns_info):
        col_name = info["col"]
        x_vals = df[col_name]

        # Random jitter in [-1, 1]
        jitter = np.random.uniform(-1, 1, size=len(df))

        # Plot each group separately
        # (so we can color them differently & have them appear in legend)
        groups = df[group_col].unique()
        for idx, grp in enumerate(groups):
            # Filter for the group
            mask = df[group_col] == grp

            fig.add_trace(
                go.Scatter(
                    x=x_vals[mask],
                    y=jitter[mask],
                    mode="markers",
                    marker=dict(color=group_color_mapping[grp]),
                    name=str(grp),
                    showlegend=True,
                    hovertemplate="<br>".join(
                        [
                            f"{col}: %{{customdata[{i}]}}"
                            for i, col in enumerate(hover_data)
                        ]
                    ),
                    customdata=df.loc[mask, hover_data].values,
                ),
                row=i + 1,
                col=1,
            )

        # Update x-axis range
        # If you provided a decreasing range, you might not need "reverse"
        # but let's handle it explicitly:
        if info["reverse"]:
            # If reversed, force the axis to go from high to low
            fig.update_xaxes(range=info["range"], row=i + 1, col=1)
        else:
            # Normal orientation
            fig.update_xaxes(range=info["range"], row=i + 1, col=1)

        # Make the subplot look "horizontal"
        fig.update_yaxes(
            range=[-1.2, 1.2],
            showticklabels=False,  # optional: hide y tick labels
            zeroline=False,
            title="",
            row=i + 1,
            col=1,
        )

    fig.update_layout(
        height=300 * n_plots,
        width=800,
        title="Grade Distribution by Grading Attribute",
        legend=dict(title="Grade", yanchor="top", y=0.95, xanchor="left", x=1.02),
    )

    return fig

In [91]:
# The columns we want to plot
columns_info = [
    {"col": "GHGIntensity", "range": [0, 50], "reverse": False},
    {"col": "ElectricityMix", "range": [100, 0], "reverse": True},
    {"col": "MissingRecordsCount", "range": [-0.5, 5.5], "reverse": False},
]

group_color_mapping = {
    "A": "#00FF00",  # Bright green
    "B": "yellowgreen",
    "C": "yellow",
    "D": "orange",
    "F": "red",
}
hover_data = [
    "PropertyName",
    "DataYear",
    "ID",
    "GHGIntensity",
    "ElectricityMix",
    "MissingRecordsCount",
]

fig = create_horizontal_subplots(
    grades_df,
    columns_info=columns_info,
    group_col="AvgPercentileLetterGrade",
    group_color_mapping=group_color_mapping,
    hover_data=hover_data,
)

fig.show()

Explore overall grades a bit more.

In [92]:
building_of_interest = "Crown Hall"
grades_df[grades_df["PropertyName"] == building_of_interest]

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade,index_y,SubmittedRecordsPercentileGrade,MissingRecordsCount,SubmittedRecordsLetterGrade,AvgPercentileGrade,AvgPercentileLetterGrade,ElectricityMix
3192,2021,256419,Crown Hall,Submitted,3360 S State Street,60616,1.0000,False,DOUGLAS,College/University,...,0.2947,0.7519,F,3438,60.0000,2.0000,B,20.2506,D,0.3000


In [93]:
building_of_interest = "Keating Hall"
grades_df[grades_df["PropertyName"] == building_of_interest]

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade,index_y,SubmittedRecordsPercentileGrade,MissingRecordsCount,SubmittedRecordsLetterGrade,AvgPercentileGrade,AvgPercentileLetterGrade,ElectricityMix
3204,2021,256434,Keating Hall,Submitted,3040 S Wabash Avenue,60616,2.0000,False,DOUGLAS,College/University,...,2.9026,1.8797,F,3452,60.0000,2.0000,B,21.1278,D,2.9000


In [94]:
building_of_interest = "Merchandise Mart"
grades_df[grades_df["PropertyName"] == building_of_interest]

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade,index_y,SubmittedRecordsPercentileGrade,MissingRecordsCount,SubmittedRecordsLetterGrade,AvgPercentileGrade,AvgPercentileLetterGrade,ElectricityMix
334,2022,103656,Merchandise Mart,Submitted Data,222 Merchandise Mart Plz,60654,NaN,True,NEAR NORTH SIDE,Office,...,60.0322,80.3845,A,373,100.0000,0.0000,A,86.7007,A,43.9000


In [95]:
building_of_interest = "Marina Towers Condominium Association"
grades_df[grades_df["PropertyName"] == building_of_interest]

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade,index_y,SubmittedRecordsPercentileGrade,MissingRecordsCount,SubmittedRecordsLetterGrade,AvgPercentileGrade,AvgPercentileLetterGrade,ElectricityMix
1923,2022,239096,Marina Towers Condominium Association,Submitted Data,300 N State St,60654,3.0000,False,NEAR NORTH SIDE,Multifamily Housing,...,100.0000,100.0000,A,2106,100.0000,0.0000,A,88.1914,A,100.0000


In [96]:
# All the A buildings:
grades_df[grades_df["AvgPercentileLetterGrade"] == "A"]

,DataYear,ID,PropertyName,ReportingStatus,Address,ZIPCode,ChicagoEnergyRating,ExemptFromChicagoEnergyRating,CommunityArea,PrimaryPropertyType,...,EnergyMixWeightedPctSum,EnergyMixPercentileGrade,EnergyMixLetterGrade,index_y,SubmittedRecordsPercentileGrade,MissingRecordsCount,SubmittedRecordsLetterGrade,AvgPercentileGrade,AvgPercentileLetterGrade,ElectricityMix
7,2022,100252,NaN,Submitted Data,10426 S MICHIGAN AVE,60628,4.0000,False,ROSELAND,Multifamily Housing,...,93.3724,92.3499,A,7,100.0000,0.0000,A,91.7484,A,93.4000
13,2022,100346,NaN,Submitted Data,2609 W BELMONT AVE,60618,4.0000,False,AVONDALE,Multifamily Housing,...,41.5970,58.8858,C,14,100.0000,0.0000,A,83.4837,A,41.6000
16,2022,100393,Ford City Mall,Submitted Data,7601 S Cicero Ave,60652,4.0000,False,WEST LAWN,Retail Store,...,93.7390,92.4284,A,17,100.0000,0.0000,A,95.7500,A,93.7000
21,2022,100418,Morgan Park HS -CPS,Submitted Data,1744 W Pryor Ave,60643,4.0000,False,MORGAN PARK,K-12 School,...,53.2056,73.9898,B,22,88.8889,1.0000,A,81.9246,A,53.2000
36,2022,100729,Northern Trust - Chicago,Submitted Data,801 S Canal St,60607,4.0000,False,NEAR WEST SIDE,Office,...,93.1747,92.3107,A,43,100.0000,0.0000,A,91.0815,A,93.2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3354,2022,257000,NaN,Submitted Data,445 W Erie,60654,4.0000,False,NaN,Office,...,42.0850,59.8274,C,3698,100.0000,0.0000,A,85.7330,A,42.1000
3355,2022,257001,NaN,Submitted Data,540 W Webster,60614,4.0000,False,NaN,Multifamily Housing,...,41.9864,59.7489,C,3699,100.0000,0.0000,A,80.8814,A,42.0000
3361,2022,257035,NaN,Submitted Data,2929 W 83rd St,60652,4.0000,False,NaN,K-12 School,...,34.7323,48.0581,C,3711,100.0000,0.0000,A,82.3722,A,34.7000
3371,2022,260060,NaN,Submitted Data,2300 2320 N COMMONWEALTH AVE,60614,4.0000,False,LINCOLN PARK,Multifamily Housing,...,59.5009,79.8745,B,3727,100.0000,0.0000,A,86.9361,A,59.5000


Check that there are no NA's in grading columns

In [97]:
grades_df[grades_columns].isna().sum()

AvgPercentileGrade                 0
AvgPercentileLetterGrade           0
EnergyMixWeightedPctSum            0
EnergyMixLetterGrade               0
EnergyMixPercentileGrade           0
GHGIntensityLetterGrade            0
GHGIntensityPercentileGrade        0
MissingRecordsCount                0
SubmittedRecordsPercentileGrade    0
SubmittedRecordsLetterGrade        0
dtype: int64